# Importar Librarias y datos

In [114]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np   
import itertools

from scipy.stats import chi2_contingency, ttest_ind
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf 

In [115]:
df_train = pd.read_parquet("output_04.parquet")

In [116]:
df_train.head()

,age,gender,education,marital_status,race,is_hispanic,employment_commitment,employment_stat,wage_per_hour,working_week_per_year,...,tax_status,gains,losses,stocks_status,citizenship,migration_year,country_of_birth_own,migration_code_change_in_msa,migration_code_change_in_reg,income_above_limit
0,79,1,secondary_complete,Widowed,White,All other,Not in labor force,0,0,52,...,Head of household,0,0,292,Native,95,US,?,?,0
1,65,1,secondary_complete,Widowed,White,All other,Children or Armed Forces,0,0,0,...,Single,0,0,0,Native,94,US,unchanged,unchanged,0
2,21,0,secondary_incomplete,Never married,Black,All other,Children or Armed Forces,0,500,15,...,Single,0,0,0,Native,94,US,unchanged,unchanged,0
3,2,1,child,Never married,Asian or Pacific Islander,All other,Children or Armed Forces,0,0,0,...,Nonfiler,0,0,0,Native,94,US,unchanged,unchanged,0
4,70,0,secondary_complete,Married-civilian spouse present,White,All other,Not in labor force,0,0,0,...,Joint both 65+,0,0,0,Native,95,US,?,?,0


# **P-Valor** 

In [118]:
print("\nEstadísticas descriptivas:")
print(df_train.describe())


Estadísticas descriptivas:
                 age         gender  employment_stat  wage_per_hour  \
count  209499.000000  209499.000000    209499.000000  209499.000000   
mean       34.518728       0.519258         0.176760      55.433487   
std        22.306738       0.499630         0.555562     276.757327   
min         0.000000       0.000000         0.000000       0.000000   
25%        15.000000       0.000000         0.000000       0.000000   
50%        33.000000       1.000000         0.000000       0.000000   
75%        50.000000       1.000000         0.000000       0.000000   
max        90.000000       1.000000         2.000000    9999.000000   

       working_week_per_year  occupation_code  total_employed          gains  \
count          209499.000000    209499.000000   209499.000000  209499.000000   
mean               23.158850        11.321734        1.956067     435.926887   
std                24.397963        14.460839        2.365154    4696.359500   
min         

In [119]:
###Este bloque de código genera las estadísticas descriptivas del conjunto de datos, tales como media, mediana, desviación estándar, y valores mínimos y máximos de las columnas numéricas.

##Es útil para comprender la dispersión y distribución de los datos.

# Identificación de columnas numéricas y categóricas

En este paso se identifican las columnas numéricas y categóricas dentro del conjunto de datos. Esto es importante para aplicar correctamente los métodos de preprocesamiento, ya que los datos numéricos y categóricos requieren tratamientos diferentes.


In [121]:
# Identificamos las columnas numéricas y categóricas
numeric_cols = ['age', 'wage_per_hour', 'working_week_per_year', 'total_employed', 'gains', 'losses', 'income_above_limit']
categorical_cols = ['gender', 'education', 'marital_status', 'race', 'is_hispanic', 
                    'employment_commitment', 'employment_stat', 'industry_code_main', 
                    'occupation_code', 'household_stat', 'tax_status', 'stocks_status', 
                    'citizenship', 'country_of_birth_own', 'migration_code_change_in_as', 
                    'migration_code_change_in_reg']


# Transformación de columnas numéricas y categoricas

En este paso se implementan dos transformaciones diferentes para las columnas numéricas y categóricas utilizando pipelines de scikit-learn. Estas transformaciones se aplicarán a los datos antes de entrenar el modelo.

Para las columnas numéricas y categoricas, se define un pipeline con los siguientes pasos:

Imputer (SimpleImputer): Rellena los valores faltantes utilizando la media de cada columna numérica.

Scaler (StandardScaler): Estandariza las variables numéricas transformando sus valores para que tengan una media de 0 y una desviación estándar de 1. Esto es importante para garantizar que todas las características numéricas estén en la misma escala y que no dominen aquellas con valores más altos.

Imputer (SimpleImputer): Rellena los valores faltantes utilizando la moda (el valor más frecuente) en cada columna categórica.

OneHotEncoder: Convierte las variables categóricas en variables dummy (0 y 1) utilizando la codificación One-Hot, eliminando la primera categoría con drop='first' para evitar la multicolinealidad

In [123]:
# Importar librerías necesarias
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns  


In [131]:

# Definir columnas numéricas y categóricas

# Convertir income_above_limit a numérico antes del preprocesamiento
#df_train['income_above_limit'] = df_train['income_above_limit'].map({'Yes': 1, 'No': 0})


numeric_cols = ['age', 'wage_per_hour', 'working_week_per_year', 'total_employed', 'gains', 'losses', 'income_above_limit']
categorical_cols = ['gender', 'education', 'marital_status', 'race', 'is_hispanic', 
                    'employment_commitment', 'employment_stat', 'industry_code_main', 
                    'occupation_code', 'household_stat', 'tax_status', 'stocks_status', 
                    'citizenship', 'country_of_birth_own', 'migration_code_change_in_as', 
                    'migration_code_change_in_reg',]

# Preprocesamiento para columnas numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputación con la media
    ('scaler', StandardScaler())  # Escalado estándar
])

# Preprocesamiento para columnas categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación con el valor más frecuente
    ('onehot', OneHotEncoder(drop='first'))  # Codificación one-hot
])

# Transformador de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Confirmación de cambios
print("Lo Logramos :).")


Lo Logramos :).


In [137]:
# Ver los valores únicos de la variable
print(df_train['income_above_limit'].unique())

"""
# Mapear valores a números
df_train['income_above_limit'] = df_train['income_above_limit'].map({
    'Below limit': 0,
    'Above limit': 1
})
"""
# Verificar que la conversión fue exitosa
print(df_train['income_above_limit'].dtype)
print(df_train['income_above_limit'].value_counts())

# Eliminar posibles valores NaN resultantes de la conversión
df_train = df_train.dropna(subset=['income_above_limit'])


[0 1]
int64
income_above_limit
0    196501
1     12998
Name: count, dtype: int64


In [139]:
modelo = smf.ols(
    formula="income_above_limit ~ age + education + marital_status + employment_stat + working_week_per_year + occupation_code + is_hispanic + gender + race",
    data=df_train
).fit()

# 6. Obtener p-valores
p_values = modelo.pvalues
significant_vars = p_values[p_values < 0.05]
print(significant_vars)

Intercept                                                 0.000000e+00
education[T.child]                                        0.000000e+00
education[T.primary]                                      0.000000e+00
education[T.secondary_complete]                           0.000000e+00
education[T.secondary_incomplete]                         0.000000e+00
education[T.technical_education/university_incomplete]    0.000000e+00
education[T.university_graduate_or_higher]                0.000000e+00
marital_status[T.Married-A F spouse present]              1.650806e-02
marital_status[T.Married-civilian spouse present]         1.751532e-14
marital_status[T.Never married]                           8.657358e-21
is_hispanic[T.Central or South American]                  6.412889e-05
is_hispanic[T.Mexican (Mexicano)]                         4.858703e-04
is_hispanic[T.Mexican-American]                           1.431908e-02
is_hispanic[T.Other Spanish]                              1.430185e-02
race[T

In [140]:
col_relevantes = modelo.pvalues[modelo.pvalues<0.05]
col_relevantes

Intercept                                                 0.000000e+00
education[T.child]                                        0.000000e+00
education[T.primary]                                      0.000000e+00
education[T.secondary_complete]                           0.000000e+00
education[T.secondary_incomplete]                         0.000000e+00
education[T.technical_education/university_incomplete]    0.000000e+00
education[T.university_graduate_or_higher]                0.000000e+00
marital_status[T.Married-A F spouse present]              1.650806e-02
marital_status[T.Married-civilian spouse present]         1.751532e-14
marital_status[T.Never married]                           8.657358e-21
is_hispanic[T.Central or South American]                  6.412889e-05
is_hispanic[T.Mexican (Mexicano)]                         4.858703e-04
is_hispanic[T.Mexican-American]                           1.431908e-02
is_hispanic[T.Other Spanish]                              1.430185e-02
race[T

# **Conclusiones del análisis de los p-valores**

### La educación y el estado civil tienen un impacto significativo en los ingresos  
- Los coeficientes de varias categorías de educación, como **doctorado, maestría, licenciatura y secundaria**, tienen p-valores extremadamente bajos (próximos a 0).  
  Esto indica que el nivel educativo tiene una fuerte relación con la probabilidad de superar el límite de ingresos.  
- El estado civil también es relevante: estar **casado con cónyuge civil presente** o **nunca haberse casado** son factores significativos.  

### El género y la raza también influyen en los ingresos  
- **Ser hombre (`gender[T.Male]`)** tiene un p-valor de **0.000000**, lo que sugiere una diferencia significativa en ingresos entre hombres y mujeres.  
- La variable **`race[T.White]`** es significativa, aunque con un p-valor más alto (**0.0393**), indicando que la raza puede tener una relación con el nivel de ingresos,  
  pero es menos determinante que otras variables como educación o empleo.  

###  **Conclusión Final**  
La **educación**, el **estado civil**, el **género** y la **ocupación** son los principales factores asociados con ingresos elevados en este modelo. 


# **Vif**


In [145]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [149]:
#columnas_numericas = df_heart.select_dtypes(include=['float64', 'int64']).columns
columnas = df_train.columns[df_train.dtypes != 'object']
# Convertir índices a listas
columnas = columnas.tolist()
columnas

# Analizamos variables numericas del dataset
df_continuas = df_train[columnas]
df_continuas.columns

Index(['age', 'gender', 'employment_stat', 'wage_per_hour',
       'working_week_per_year', 'occupation_code', 'total_employed', 'gains',
       'losses', 'stocks_status', 'migration_year', 'income_above_limit'],
      dtype='object')

In [151]:
vif_acum = {}

for i in columnas:
   
    x = df_continuas.loc[:,df_continuas.columns != i]
    y = df_continuas.loc[:,i]

    lm = LinearRegression()
    lm.fit(x, y)
    r2 = lm.score(x, y)

    vif = (1/(1-r2))
    vif_acum[i] = vif

vif_acum

{'age': 1.0742732461465767,
 'gender': 1.0611866623393869,
 'employment_stat': 1.0918333026428886,
 'wage_per_hour': 1.0523117969479616,
 'working_week_per_year': 3.1731899737573777,
 'occupation_code': 1.903715536775807,
 'total_employed': 2.302575955716003,
 'gains': 1.0750704635161745,
 'losses': 1.034489420359212,
 'stocks_status': 1.0524702807763373,
 'migration_year': 1.0009750548293233,
 'income_above_limit': 1.2731050989006856}

## Conclusiones del Análisis de VIF

### No hay signos fuertes de multicolinealidad en la mayoría de las variables  
- La mayoría de los valores de **VIF** están por debajo de **5**, lo que indica que las variables no presentan una colinealidad significativa.  
- Variables como **age (1.06)**, **wage_per_hour (1.05)**, **gains (1.03)** y **losses (1.02)** tienen valores muy cercanos a **1**, lo que significa que prácticamente no están correlacionadas con otras variables.  
- La variable **migration_year** tiene el VIF más bajo (**1.00**), lo que indica que no está influenciada por ninguna otra variable en el conjunto de datos.  

### Posible colinealidad moderada en *working_week_per_year*, *total_employed* y *occupation_code*  
- **working_week_per_year** tiene un **VIF de 3.03**, el más alto del conjunto, lo que sugiere que puede tener cierta correlación con otras variables, aunque sigue en un rango aceptable.  
- **total_employed (2.28)** y **occupation_code (1.79)** también tienen valores algo elevados en comparación con otras variables, lo que podría indicar que comparten información con otras características del modelo.  
- Aunque estos valores no son alarmantes, si en un análisis más profundo se detecta impacto en la interpretación del modelo, se podría considerar eliminar una de estas variables o combinar información redundante.  

